### Create database and insert records from csv files

In [2]:
import sqlite3
import pandas as pd 
from tabulate import tabulate

# Connect to the SQLite database
connection = sqlite3.connect('hospital.db')
cursor = connection.cursor()

# Delete tables if exist  
cursor.execute(''' DROP TABLE IF EXISTS patients;  ''')
cursor.execute(''' DROP TABLE IF EXISTS doctors;  ''')
cursor.execute(''' DROP TABLE IF EXISTS admissions;  ''')
cursor.execute(''' DROP TABLE IF EXISTS province_names;  ''')

# Create tables based on the schema 
cursor.execute(''' CREATE TABLE patients (
                    patient_id INTEGER PRIMARY KEY,
                    first_name VARCHAR(30),
                    last_name VARCHAR(30),
                    gender CHAR(1),
                    birth_date DATE,
                    city VARCHAR(30),
                    province_id CHAR(2),
                    allergies VARCHAR(80),
                    height DECIMAL(3,0),
                    weight DECIMAL(4,0)    ); ''')

cursor.execute(''' CREATE TABLE doctors (
                    doctor_id INTEGER PRIMARY KEY,
                    first_name VARCHAR(30),
                    last_name VARCHAR(30),
                    speciality VARCHAR(25)   ); ''')

cursor.execute(''' CREATE TABLE admissions (
                    patient_id INT,
                    admission_date DATE,
                    discharge_date DATE,
                    diagnosis VARCHAR(50),
                    attending_doctor_id INT,
                    FOREIGN KEY(patient_id) REFERENCES patients(patient_id),
                    FOREIGN KEY(attending_doctor_id) REFERENCES doctors(doctor_id)    );  ''')

cursor.execute(''' CREATE TABLE province_names (
                    province_id CHAR(2) PRIMARY KEY,
                    province_name VARCHAR(30)    );  ''')

# Load data from CSV files into tables
# Load patients data
patients_df = pd.read_csv(r'C:\Volume\Projects\Patient Care and Admission Pattern Analysis\csv\patients.csv')
patients_df.to_sql('patients', connection, if_exists='append', index=False)

# Load doctors data
doctors_df = pd.read_csv(r'C:\Volume\Projects\Patient Care and Admission Pattern Analysis\csv\doctors.csv')
doctors_df.to_sql('doctors', connection, if_exists='append', index=False)

# Load admissions data
admissions_df = pd.read_csv(r'C:\Volume\Projects\Patient Care and Admission Pattern Analysis\csv\admissions.csv')
admissions_df.to_sql('admissions', connection, if_exists='append', index=False)

# Load province names data
province_names_df = pd.read_csv(r'C:\Volume\Projects\Patient Care and Admission Pattern Analysis\csv\province_names.csv')
province_names_df.to_sql('province_names', connection, if_exists='append', index=False)


13

### Functions

In [3]:
# Display table from hospital.db 
def showTableFromDB(table_name):
    query = f'SELECT * FROM {table_name} '
    cursor.execute(query)

    # Displaying first 5 records
    tableRecords = cursor.fetchmany(5)
    
    column_name = [ description[0] for description in cursor.description ]
    print( tabulate(tableRecords, headers= column_name, tablefmt= 'simple') )

In [4]:
# Display result of query
def displayOutput(cursor):
    result = cursor.fetchall()

    print('Number of Records - ', len(result) )

    # Fetch column names
    column_name = [ description[0] for description in cursor.description ]

    # Print the result in table format
    print( tabulate(result, headers= column_name, tablefmt= 'simple') )

### Questions from Management Team :

#### 1. Can you provide the first and last names of all male patients ?

In [5]:
cursor.execute(''' SELECT first_name, last_name, gender 
                    FROM patients 
                    WHERE gender LIKE 'M'; ''')

displayOutput(cursor)

Number of Records -  2468
first_name     last_name         gender
-------------  ----------------  --------
Donald         Waterfield        M
Mickey         Baasha            M
Jiji           Sharma            M
Blair          Diaz              M
Charles        Wolfe             M
Thomas         ONeill            M
Sonny          Beckett           M
Cedric         Coltrane          M
Hank           Spencer           M
Rick           Bennett           M
Woody          Bashir            M
Tom            Halliwell         M
John           West              M
Jon            Doggett           M
Nino           Andrews           M
John           Farley            M
Jim            Thomas            M
Roderick       Payne             M
Sam            Threep            M
Roland         Edge              M
Don            Wiggum            M
Anthony        Maxwell           M
James          Sullivan          M
Charles        Andonuts          M
Don            Edwards           M
Stone          Cu

#### 2. Who are the patients that do not have any allergies recorded ?

In [6]:
cursor.execute(''' SELECT first_name, last_name FROM patients
                    WHERE allergies IS NULL; ''')

displayOutput(cursor)

Number of Records -  2059
first_name     last_name
-------------  ----------------
Donald         Waterfield
Blair          Diaz
Thomas         ONeill
Sonny          Beckett
Cedric         Coltrane
Hank           Spencer
Sara           di Marco
Amy            Leela
Rachel         Winterbourne
John           West
Jon            Doggett
Angel          Edwards
Brodie         Beck
Nea            Kobayakawa
Laura          Halliwell
Lucille        Maureen
Roderick       Payne
Bertha         Crowley
Don            Wiggum
Anthony        Maxwell
Anne           Marlowe
Charles        Andonuts
Don            Edwards
Cross          Gordon
Harry          Kuramitsu
Gary           Ramotswe
Phil           Beckett
Gala           Littlefield
Gregory        James
Molly          Jackson
Rose           Spellman
Will           Manu
Rocky          Donahue
Mary           Spellman
Didi           Dean
Lily           Crusher
Billy          Jones
Vicki          Babbling
Temple         Marlowe
Jackie         Loveg

#### 3. Could you list the patients whose names start with the letter 'C' ?

In [7]:
cursor.execute(''' SELECT * FROM patients
                    WHERE first_name LIKE 'C%';  ''')

displayOutput(cursor) 

Number of Records -  302
  patient_id  first_name    last_name         gender    birth_date    city            province_id    allergies                   height    weight
------------  ------------  ----------------  --------  ------------  --------------  -------------  ------------------------  --------  --------
           5  Charles       Wolfe             M         2017-11-19    Orillia         ON             Penicillin                      47        10
          10  Cedric        Coltrane          M         1961-11-10    Toronto         ON                                            157        61
          39  Charles       Andonuts          M         2016-05-20    Hamilton        ON                                             62        15
          45  Cross         Gordon            M         2009-03-20    Ancaster        ON                                            125        53
          63  Calleigh      Dean              F         2003-07-06    Hamilton        ON           

#### 4. Can we get a list of patients weighing between 100 and 120 units ?

In [8]:
cursor.execute(''' SELECT first_name, last_name 
                    FROM patients
                    WHERE weight BETWEEN 100 AND 120;  ''')

displayOutput(cursor)

Number of Records -  952
first_name    last_name
------------  ----------------
Jiji          Sharma
Blair         Diaz
Thomas        ONeill
Sonny         Beckett
Tom           Halliwell
Jon           Doggett
Angel         Edwards
John          Farley
Temple        Russert
Don           Edwards
Gary          Ramotswe
Phil          Chester
Woody         Riviera
Rocky         Donahue
Shiro         Whale
Didi          Dean
Catherine     Minoru
Danny         Hardbroom
Lily          Vargas
Reginald      Jackson
Phoebe        Bell
Ann           Blocksberg
Sam           Spratt
Prue          Cleever
Zoe           Miller
Mendy         Strate
Hal           Riviera
John          Stape
Alice         Taylor
Salvatore     Neutron
Thomas        Fortune
Arnold        Love
Vin           Smyth
Kenny         Skelton
Mike          Madison
Keziah        Brearly
Jeff          Tyrell
Daneel        Carpenter
Robert        Page
Wanda         Redfield
Brooke        Bowles
Hoshi         Johnson
Joanna        Coo

#### 5. Could you update records so that patients without allergy information show as 'NKA' (No Known Allergies) ?

In [9]:
cursor.execute(''' UPDATE patients
                    SET allergies = 'NKA'
                    WHERE allergies IS NULL; ''')

# Saving changes in databse
connection.commit()

showTableFromDB('patients') 

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


#### 6. Can we display the full name of each patient ?  

In [10]:
cursor.execute(''' SELECT CONCAT(first_name, ' ', last_name) AS full_name
                    FROM patients;  ''')

displayOutput(cursor)

Number of Records -  4530
full_name
---------------------------
Donald Waterfield
Mickey Baasha
Jiji Sharma
Blair Diaz
Charles Wolfe
Sue Falcon
Thomas ONeill
Sonny Beckett
Sister Spitzer
Cedric Coltrane
Hank Spencer
Sara di Marco
Daphne Seabright
Rick Bennett
Amy Leela
Woody Bashir
Tom Halliwell
Rachel Winterbourne
John West
Jon Doggett
Angel Edwards
Brodie Beck
Beanie Foster
Nino Andrews
John Farley
Nea Kobayakawa
Laura Halliwell
Lucille Maureen
Jim Thomas
Roderick Payne
Sam Threep
Bertha Crowley
Roland Edge
Don Wiggum
Anthony Maxwell
James Sullivan
Anne Marlowe
Kelly Hamilton
Charles Andonuts
Temple Russert
Don Edwards
Alice Donovan
Stone Cutting
Neil Allan
Cross Gordon
Phoebe Bigelow
Harry Kuramitsu
Gary Ramotswe
Phil Chester
Phil Beckett
Gala Littlefield
Gregory James
Bruno Wolfe
Molly Jackson
Rose Spellman
Will Manu
Woody Riviera
Georgette Gallagher
Rocky Donahue
Roland Murphy
Mary Spellman
Shiro Whale
Calleigh Dean
Didi Dean
Haiku Alpert
Joseph Justin
Catherine Minoru
Danny Hardb

#### 7. Could we see each patient’s full name along with their full province name ?

In [11]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [12]:
showTableFromDB('province_names')

province_id    province_name
-------------  -------------------------
AB             Alberta
BC             British Columbia
MB             Manitoba
NB             New Brunswick
NL             Newfoundland and Labrador


In [13]:
cursor.execute(''' SELECT first_name, last_name, province_name
                    FROM patients as p 
                    INNER JOIN province_names as pn 
                    ON p.province_id = pn.province_id ;  ''')

displayOutput(cursor)

Number of Records -  4530
first_name     last_name         province_name
-------------  ----------------  -------------------------
Donald         Waterfield        Ontario
Mickey         Baasha            Ontario
Jiji           Sharma            Ontario
Blair          Diaz              Ontario
Charles        Wolfe             Ontario
Sue            Falcon            Ontario
Thomas         ONeill            Ontario
Sonny          Beckett           Nova Scotia
Sister         Spitzer           Ontario
Cedric         Coltrane          Ontario
Hank           Spencer           Ontario
Sara           di Marco          Ontario
Daphne         Seabright         Ontario
Rick           Bennett           Ontario
Amy            Leela             Ontario
Woody          Bashir            Ontario
Tom            Halliwell         Ontario
Rachel         Winterbourne      Ontario
John           West              Ontario
Jon            Doggett           Ontario
Angel          Edwards           Ontario
Bro

#### 8. How many of our patients were born in 2010 ?

In [14]:
cursor.execute(''' SELECT count(birth_date) AS Count
                    FROM patients 
                    WHERE strftime('%Y', birth_date) = '2010';  ''')

displayOutput(cursor)

Number of Records -  1
  Count
-------
     55


#### 9. Find the tallest patient, showing their full name and height.

In [15]:
cursor.execute(''' SELECT first_name, last_name, MAX(height)
                    FROM patients ''')

displayOutput(cursor)

Number of Records -  1
first_name    last_name      MAX(height)
------------  -----------  -------------
Sam           Haruko                 226


#### 10. Could you retrieve all the details for patients with IDs 1, 45, 534, 879, and 1000 ?

In [16]:
cursor.execute(''' SELECT * 
                    FROM patients
                    WHERE patient_id IN (1, 45, 534, 879, 1000);  ''')

displayOutput(cursor)

Number of Records -  5
  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
          45  Cross         Gordon       M         2009-03-20    Ancaster  ON             NKA               125        53
         534  Don           Zatara       M         2008-01-11    Timmins   ON             NKA               136        67
         879  Orla          Shawn        F         1967-09-24    Sarnia    ON             Penicillin        149        65
        1000  Rick          Williams     M         1975-04-13    Hamilton  ON             Penicillin        176       127


#### 11. What is the total count of admissions to date ?

In [17]:
cursor.execute(''' SELECT count(*) AS total_admissions
                    FROM admissions;  ''')

displayOutput(cursor)

Number of Records -  1
  total_admissions
------------------
              5067


#### 12. Who were admitted and discharged on the same day ?

In [18]:
cursor.execute(''' SELECT *
                    FROM admissions
                    WHERE admission_date = discharge_date;  ''')

displayOutput(cursor)

Number of Records -  481
  patient_id  admission_date    discharge_date    diagnosis                                             attending_doctor_id
------------  ----------------  ----------------  --------------------------------------------------  ---------------------
           1  2018-09-20        2018-09-20        Ineffective Breathin Pattern R/T Fluid Accumulatio                     24
           9  2018-12-31        2018-12-31        Ruptured Appendicitis                                                  19
          10  2019-02-27        2019-02-27        Lower Quadrant Pain                                                    27
          17  2019-03-04        2019-03-04        Diabetes Mellitus                                                       9
          28  2019-03-30        2019-03-30        Cancer Of The Stomach                                                  26
          31  2018-09-26        2018-09-26        Cardiovascular Disease                                   

#### 13. How many admissions has patient ID 579 had so far ?

In [19]:
cursor.execute(''' SELECT patient_id, COUNT(*) AS total_admissions
                    FROM admissions
                    WHERE patient_id = 579;  ''')

displayOutput(cursor)

Number of Records -  1
  patient_id    total_admissions
------------  ------------------
         579                   2


#### 14. Could you list the unique cities in Nova Scotia where our patients live ?

In [20]:
cursor.execute(''' SELECT DISTINCT city AS unique_cities
                    FROM patients
                    WHERE province_id = 'NS';  ''')

displayOutput(cursor)

Number of Records -  3
unique_cities
---------------
Port Hawkesbury
Halifax
Inverness


#### 15. Could you find patients First name, Last name, Birth Date who are over 160 cm in height and over 70 kg in weight ?

In [21]:
cursor.execute(''' SELECT first_name, last_name, birth_date
                    FROM patients
                    WHERE height > 160 AND weight > 70;  ''')

displayOutput(cursor)

Number of Records -  2091
first_name     last_name         birth_date
-------------  ----------------  ------------
Mickey         Baasha            1981-05-28
Jiji           Sharma            1957-09-05
Blair          Diaz              1967-01-07
Thomas         ONeill            1993-01-31
Sonny          Beckett           1952-12-11
Sister         Spitzer           1966-10-15
Rick           Bennett           1977-01-27
Amy            Leela             1977-06-25
Tom            Halliwell         1987-08-01
Rachel         Winterbourne      1966-04-26
Jon            Doggett           1951-12-25
Angel          Edwards           1975-08-22
Nino           Andrews           2001-04-21
John           Farley            1956-09-12
Lucille        Maureen           1953-08-22
Sam            Threep            1982-06-16
Roland         Edge              1967-09-25
Don            Wiggum            2002-11-15
Anne           Marlowe           1996-09-17
Temple         Russert           1953-05-11
Don 

#### 16. Who in Hamilton has recorded allergies ?

In [26]:
cursor.execute(''' SELECT first_name, last_name, allergies
                    FROM patients
                    WHERE city = 'Hamilton' AND allergies != 'NKA';  ''')

displayOutput(cursor)

Number of Records -  1046
first_name    last_name         allergies
------------  ----------------  ------------------------
Jiji          Sharma            Penicillin
Tom           Halliwell         Ragweed
Nino          Andrews           Peanuts
John          Farley            Gluten
Sam           Threep            Sulpha
Temple        Russert           Hay Fever
Alice         Donovan           Penicillin
Stone         Cutting           Codeine
Phil          Chester           Penicillin
Roland        Murphy            Sulfa Drugs
Shiro         Whale             Penicillin
Calleigh      Dean              Penicillin
Lily          Vargas            Codeine
Kazuki        Ranall            Penicillin
Mike          Mars              Penicillin
Matt          Goergen           Penicillin
Ann           Blocksberg        Penicillin
Zoe           Miller            Honey
Gene          Edwards           Sulphate
Emmett        Erin              Penicillin
Olwyn         Chino             Tetracycli

#### 17. List all unique patient birth years in ascending order.

In [34]:
cursor.execute(''' SELECT DISTINCT strftime('%Y', birth_date) AS birth_year
                    FROM patients
                    ORDER BY birth_date ASC;  ''')

displayOutput(cursor)

Number of Records -  93
  birth_year
------------
        1918
        1923
        1925
        1926
        1927
        1928
        1929
        1931
        1933
        1934
        1936
        1937
        1938
        1939
        1940
        1941
        1942
        1943
        1944
        1945
        1946
        1947
        1948
        1949
        1950
        1951
        1952
        1953
        1954
        1955
        1956
        1957
        1958
        1959
        1960
        1961
        1962
        1963
        1964
        1965
        1966
        1967
        1968
        1969
        1970
        1971
        1972
        1973
        1974
        1975
        1976
        1977
        1978
        1979
        1980
        1981
        1982
        1983
        1984
        1985
        1986
        1987
        1988
        1989
        1990
        1991
        1992
        1993
        1994
        1995
        1996
        1997
        1998
 

In [30]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


#### 18. Are there any names that appear only once in our patient records ? For example, if two or more people are named 'John' in the first_name column then don't include their name in the output list. If only 1 person is named 'Leo' then include them in the output.

In [38]:
cursor.execute(''' SELECT first_name
                    FROM patients
                    GROUP BY first_name
                    HAVING COUNT(first_name) = 1;  ''')

displayOutput(cursor)

Number of Records -  319
first_name
------------
Abby
Adelaide
Adelia
Akira
Albert
Aldo
Alec
Alicia
Allan
Alpa
Amane
Amitabh
Angel
Angelina
Angus
Anne
Anton
Antonia
Anubis
Apolline
April
Ariel
Asa
Astrid
Asuka
Audrey
Aurora
Avery
Bathilda
Bathsheda
Beverly
Bianca
Bibi
Bigby
Bob
Borat
Brick
Brick Top
Brodie
Buffy
Buford
Bunk
Burr
Byron
Caitlin
Calvin
Carla
Carlton
Carmel
Carmen
Cary
Cat
Chan
Cherry
China
Cho
Christina
Christy
Chuny
Cindy
Clancy
Clarence
Claudette
Clay
Clayton
Colleen
Connie
Cooper
Crematia
Cristina
Cully
Cuthbert
Cyrus
Daisuke
Daisy
Dave
Davina
Debbie
Debra
Delia
Denise
Deunan
Diana
Dickie
Dion
Dorian
Drogulous
Duncan
Dusty
Dwight
Edgar
Eglantine
Eleanor
Ellie
Ellis
Elly
Elsie
Emerson
Enid
Epiphany
Eric
Erica
Erika
Ervin
Escobar
Fancy
Frankie
Frau
Frederico
Gabriel
Gabrielle
Gemma
Georgia
Gina
Ginevra
Gordon
Gus
Gwenog
Gytha
Haley
Hans
Hari
Harvey
Henri
Herb
Hermione
Hildegarde
Hillary
Hiroshi
Holi
Holland
Horatio
Ichabod
Isaac
Isabel
Isabella
Jabba
Janine
Janus
Jasmine

#### 19. Could you find patients whose first names start and end with ‘S’ and have at least six letters ?

In [45]:
cursor.execute(''' SELECT patient_id, first_name
                    FROM patients
                    WHERE first_name LIKE 'S%S' AND LENGTH(first_name) >= 6;  ''')

displayOutput(cursor)

Number of Records -  11
  patient_id  first_name
------------  ------------
         496  Spiros
         629  Spiros
         648  Stanislaus
        1273  Stanislaus
        1789  Seamus
        1926  Stanislaus
        1996  Stanislaus
        2258  Spiros
        2378  Stanislaus
        2771  Stanislaus
        3487  Stanislaus


#### 20. Can we list patients diagnosed with ‘Dementia’ ?

In [46]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [48]:
showTableFromDB('admissions')

  patient_id  admission_date    discharge_date    diagnosis                                             attending_doctor_id
------------  ----------------  ----------------  --------------------------------------------------  ---------------------
           1  2018-11-06        2018-11-08        Ovarian Dermoid-Cyct                                                   21
           1  2018-09-20        2018-09-20        Ineffective Breathin Pattern R/T Fluid Accumulatio                     24
           3  2019-01-24        2019-01-29        Cardiac Arrest                                                          2
           3  2018-10-21        2018-10-27        Congestive Heart Failure                                                8
           6  2018-06-13        2018-06-15        Asthma Exacerbation                                                     3


In [52]:
cursor.execute(''' SELECT p.patient_id, p.first_name, p.last_name
                    FROM patients AS p
                    LEFT JOIN admissions AS a 
                    ON p.patient_id = a.patient_id
                    WHERE diagnosis = 'Dementia';  ''')

displayOutput(cursor)

Number of Records -  26
  patient_id  first_name    last_name
------------  ------------  -----------
         160  Miranda       Delacour
         178  David         Bustamonte
         207  Matt          Celine
         613  Jaki          Granger
         836  Montana       Vimes
         924  Simon         Spellman
        1201  Irene         Murphy
        1264  Jillian       Valentine
        1402  Kathryn       Hallow
        1491  Doris         McGrew
        1585  Alex          Cantropus
        1749  Alejandro     Mellie
        1798  Sister        Trenton
        1844  Jadu          Wilson
        1938  Anthony       Anneke
        2292  Jessica       Talbot
        2532  Lynn          Willing
        2689  Kurt          Jones
        2834  Juliet        Harkness
        3375  Agnes         Barkley
        3496  Harry         Lucas
        3555  Jack          Caspar
        3842  Claudia       Drew
        4256  Rita          Wolfe
        4357  Alex          Moss
        440

#### 21. List all patient first name ordered by name length, then alphabetically.

In [55]:
cursor.execute(''' SELECT first_name 
                    FROM patients
                    ORDER BY LENGTH(first_name) ASC, first_name ASC;  ''')

displayOutput(cursor)

Number of Records -  4530
first_name
-------------
Al
Al
Al
Al
Al
Al
Al
Bo
Bo
Bo
Bo
Bo
Bo
Cy
Cy
Ed
Ed
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Jo
Ri
Te
Te
Abe
Abe
Abe
Abe
Abe
Abi
Abi
Abi
Aki
Aki
Aki
Aki
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Amy
Ana
Ana
Ana
Ana
Ana
Ana
Ann
Ann
Ann
Ann
Ann
Ann
Ann
Art
Art
Art
Art
Art
Art
Art
Art
Art
Asa
Aya
Aya
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Ben
Bob
Cal
Cal
Cal
Cal
Cal
Cat
Cho
Dan
Dan
Dan
Dan
Dan
Dol
Dol
Dol
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Don
Ema
Ema
Ema
Eva
Eva
Eva
Eva
Eva
Eva
Eva
Eve
Eve
Eve
Fin
Fin
Fin
Fiz
Fiz
Fiz
Fiz
Fiz
Gil
Gil
Gil
Gus
Guy
Guy
Guy
Guy
Guy
Guy
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Hal
Ian
Ian
Ian
Ivy
Ivy
Ivy
Jay
Jay
Jay
Jay
Jay
Jay
Jay
Jay
Jay
Jay
Jen
Jen
Jen
Jim
Jim
Jim
Jim
Jim
Jim
Jim
Jim
Jim
Jim
Jin
Jin
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Joe
Jon
Jon
Jon
Jon
Ka

#### 22. What is the count of male and female patients ? Display them side by side.

In [67]:
cursor.execute(''' SELECT 
                    SUM(gender = 'M') AS male_count,
                    SUM(gender = 'F') AS female_count
                    FROM patients;  ''')

displayOutput(cursor)

Number of Records -  1
  male_count    female_count
------------  --------------
        2468            2062


#### 23. List patients allergic to Penicillin or Morphine, ordered by allergy type and name.

In [70]:
cursor.execute(''' SELECT first_name, last_name, allergies
                    FROM patients
                    WHERE allergies IN ('Penicillin', 'Morphine')
                    ORDER BY allergies ASC, first_name ASC, last_name ASC;  ''')

displayOutput(cursor)

Number of Records -  1104
first_name    last_name         allergies
------------  ----------------  -----------
Briareos      Hayes             Morphine
Christine     Argyros           Morphine
Griselda      Hopper            Morphine
Henry         Huang             Morphine
Janice        Redfield          Morphine
Jesse         Guarnaccia        Morphine
Joel          Takata            Morphine
Jon           Guarnaccia        Morphine
Juan          Davies            Morphine
Kelly         Landsman          Morphine
Kent          Jacobs            Morphine
Mickey        Yamamoto          Morphine
Norman        Scott             Morphine
Patricia      Newbury           Morphine
Patty         Brady             Morphine
Raylan        Mallory           Morphine
Sarah         Pike              Morphine
Sarah Jane    Jones             Morphine
Temple        Starsmore         Morphine
Tugg          Travis            Morphine
Tyra          Shannon           Morphine
Uri           Falcon       

#### 24. Are there any patients admitted multiple times for the same diagnosis ?

In [75]:
cursor.execute(''' SELECT patient_id, diagnosis 
                    FROM admissions 
                    GROUP BY patient_id, diagnosis
                    HAVING COUNT(*) > 1;  ''')

displayOutput(cursor)

Number of Records -  11
  patient_id  diagnosis
------------  ---------------------------------
         137  Pregnancy
         320  Pneumonia
        1577  Congestive Heart Failure
        2004  Left Shoulder Rotator Cuff Repair
        2859  Severed Spine At C3
        3012  Appendicitis
        3367  Pyelonephritis
        3468  Congestive Heart Failure
        4083  Congestive Heart Failure
        4121  Congestive Heart Failure
        4363  Congestive Heart Failure


#### 25. List of city and the total number of patients in the city. Order from most to least patients and then by city name ascending.

In [91]:
cursor.execute(''' SELECT city, COUNT(*) AS total_number_of_patients
                    FROM patients
                    GROUP BY city
                    ORDER BY COUNT(*) DESC, city ASC;  ''')

displayOutput(cursor)

Number of Records -  93
city               total_number_of_patients
---------------  --------------------------
Hamilton                               1938
Toronto                                 317
Burlington                              276
Brantford                               147
Ancaster                                117
Stoney Creek                            107
Cambridge                                79
Dundas                                   79
Milton                                   65
Paris                                    58
Grimsby                                  55
Timmins                                  53
Ottawa                                   51
Halifax                                  50
Oakville                                 44
Sarnia                                   43
Fruitland                                42
Barrie                                   39
Delhi                                    37
St. Catharines                           35
Walnut G

In [ ]:
cursor.execute(''' SELECT *
                    FROM patients;  ''')

displayOutput(cursor)

#### 26. Can we show everyone affiliated with the hospital as either ‘Patient’ or ‘Doctor’ ?

In [92]:
showTableFromDB('patients')

  patient_id  first_name    last_name    gender    birth_date    city      province_id    allergies      height    weight
------------  ------------  -----------  --------  ------------  --------  -------------  -----------  --------  --------
           1  Donald        Waterfield   M         1963-02-12    Barrie    ON             NKA               156        65
           2  Mickey        Baasha       M         1981-05-28    Dundas    ON             Sulfa             185        76
           3  Jiji          Sharma       M         1957-09-05    Hamilton  ON             Penicillin        194       106
           4  Blair         Diaz         M         1967-01-07    Hamilton  ON             NKA               191       104
           5  Charles       Wolfe        M         2017-11-19    Orillia   ON             Penicillin         47        10


In [93]:
showTableFromDB('doctors')

  doctor_id  first_name    last_name    speciality
-----------  ------------  -----------  -------------------------
          1  Claude        Walls        Internist
          2  Joshua        Green        Cardiologist
          3  Miriam        Tregre       General Surgeon
          4  James         Russo        Obstetrician/Gynecologist
          5  Scott         Hill         Gastroenterologist


In [102]:
cursor.execute(''' SELECT first_name, last_name, 'Patient' AS role FROM patients
                    UNION ALL
                   SELECT first_name, last_name, 'Doctor' AS role FROM doctors;  ''')

displayOutput(cursor)

Number of Records -  4557
first_name     last_name         role
-------------  ----------------  -------
Donald         Waterfield        Patient
Mickey         Baasha            Patient
Jiji           Sharma            Patient
Blair          Diaz              Patient
Charles        Wolfe             Patient
Sue            Falcon            Patient
Thomas         ONeill            Patient
Sonny          Beckett           Patient
Sister         Spitzer           Patient
Cedric         Coltrane          Patient
Hank           Spencer           Patient
Sara           di Marco          Patient
Daphne         Seabright         Patient
Rick           Bennett           Patient
Amy            Leela             Patient
Woody          Bashir            Patient
Tom            Halliwell         Patient
Rachel         Winterbourne      Patient
John           West              Patient
Jon            Doggett           Patient
Angel          Edwards           Patient
Brodie         Beck              P

#### 27. Could you show common allergy types, removing ‘NKA’ and unknown entries?

In [ ]:
cursor.execute(''' SELECT *
                    FROM patients;  ''')

displayOutput(cursor)

#### 28. Could we get a list of patients born in the 1970s, starting from the earliest birth date?

In [ ]:
cursor.execute(''' SELECT *
                    FROM patients;  ''')

displayOutput(cursor)

#### 29. Can we display patient names in ‘LASTNAME, firstname’ format, sorted by first name?

In [ ]:
cursor.execute(''' SELECT *
                    FROM patients;  ''')

displayOutput(cursor)

#### 30. Which provinces have a combined patient height of 7,000 or more?

In [ ]:
cursor.execute(''' SELECT *
                    FROM patients;  ''')

displayOutput(cursor)

#### 30. Which provinces have a combined patient height of 7,000 or more?

In [ ]:
cursor.execute(''' SELECT 
                    FROM ;  ''')

displayOutput(cursor)


#### 31. What’s the weight difference range for patients with the last name ‘Maroni’?
#### 32. Can we see which day of the month has the most admissions?
#### 33. What are the details for the latest admission of patient ID 542?
#### 34. Can we see admissions meeting certain criteria, like patients with odd-numbered IDs and specific doctor IDs?
#### 35. For each doctor, how many admissions have they attended to date?
#### 36. Could we display each doctor’s name and the first and last admission dates they attended?
#### 37. What is the count of patients from each province? Order by the highest count.
#### 38. Could you list each admission with the patient’s name, diagnosis, and doctor’s name?
#### 39. Are there any duplicate patients based on first and last names?
#### 40. Can we display each patient’s name, height (in feet), weight (in pounds), birth date, and full gender name?
#### 41. Which patients don’t have any admission records?
#### 42. Could we categorize patients by weight groups? Show the count of patients in each group.
#### 43. Can you identify patients who are classified as obese, and show their weight, height, and obesity status?
#### 44. Could you show patients diagnosed with ‘Dementia’ by Dr. Lisa and include her specialty?
#### 45. Show the temporary password for patients after their first admission, formatted with specific details.
#### 46. How much do patients with and without insurance pay for admissions on average?
#### 47. Which provinces have more male than female patients? Show the province names.
#### 48. Could we find the patient with certain criteria, including birth month, weight range, and city?
#### 49. What percent of our patients are male, rounded to the nearest whole number?
#### 50. Could you display daily admissions along with any change from the previous day?
#### 51. List all province names alphabetically, placing Ontario at the top.
#### 52. How many admissions does each doctor handle annually? Show by doctor and specialty.

### Close databse connection

In [23]:
# Close the connection
#connection.close()